In [12]:
import mne
import numpy as np
import double_dipper
from double_dipper.io import filePairs, partition

In [8]:
divider = lambda meta: meta["id"]
def labeller(meta):
    strat = meta["strategy"]
    if strat is None: return None
    
    if strat.lower().startswith("fact"):        return 0
    elif strat.lower().startswith("procedure"): return 1
    else:                                       return None

In [9]:
subjNos = [i for i in range(1,11+1) if i != 5]
pairs = filePairs(*[f"cleaned/main/{i}" for i in subjNos])
dataset = partition(divider, labeller, pairs)

In [10]:
list(dataset.keys())

[1, 2, 3, 4, 6, 7, 8, 9, 10, 11]

In [14]:
sfreq = double_dipper.constants.sfreq
for (subjNo, fold) in dataset.items():
    data = fold["x"]
    data = mne.filter.filter_data(data, sfreq, l_freq=1, h_freq=32)
    data = data.reshape([data.shape[0], np.prod(data.shape[1:])])
    fold["x"] = data

Setting up band-pass filter from 1 - 32 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 32.00 Hz
- Upper transition bandwidth: 8.00 Hz (-6 dB cutoff frequency: 36.00 Hz)
- Filter length: 845 samples (3.301 sec)

Setting up band-pass filter from 1 - 32 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 32.00 Hz
- Upper transition bandwidth: 8.00 Hz (-6 dB cutoff frequency: 36.00 Hz)
- Fi

In [24]:
numFolds = len(dataset.keys())
precision = np.zeros(numFolds)
recall = np.zeros(numFolds)
f1 = np.zeros(numFolds)

In [25]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import precision_recall_fscore_support

In [26]:
subjNos = list(dataset.keys())
for (i, testNo) in enumerate(subjNos):
    testX = dataset[testNo]["x"]
    testY = dataset[testNo]["y"]
    others = [dataset[no] for no in subjNos if no != testNo]
    X = np.concatenate( list(map(lambda part: part["x"], others)) , axis = 0 )
    Y = np.concatenate( list(map(lambda part: part["y"], others)), axis = 0)
    model = LinearDiscriminantAnalysis()
    model.fit(X, Y)
    preds = model.predict(testX)
    (precision[i], recall[i], f1[i], _) = precision_recall_fscore_support(testY, preds, average="binary")
    print(f"Held out subject {testNo}: f1 = {f1[i]}")
    

/home/elmines/git/DoubleDipperAnalysis/env/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


Held out subject 1: f1 = 0.28571428571428575


/home/elmines/git/DoubleDipperAnalysis/env/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


Held out subject 2: f1 = 0.21428571428571427


/home/elmines/git/DoubleDipperAnalysis/env/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


...

/home/elmines/git/DoubleDipperAnalysis/env/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


Held out subject 9: f1 = 0.5102040816326532


/home/elmines/git/DoubleDipperAnalysis/env/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


Held out subject 10: f1 = 0.41791044776119407
Held out subject 11: f1 = 0.3466666666666667


/home/elmines/git/DoubleDipperAnalysis/env/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [27]:
precision

array([0.26666667, 0.23076923, 0.2972973 , 0.31428571, 0.51851852,
       0.02941176, 0.25      , 0.55555556, 0.53846154, 0.26530612])

In [29]:
recall

array([0.30769231, 0.2       , 0.37931034, 0.26190476, 0.35      ,
       0.2       , 0.30434783, 0.47169811, 0.34146341, 0.5       ])